In [1]:
from dotenv import load_dotenv
import os
print("OPENAI_API_KEY", os.getenv("OPENAI_API_KEY")[:10])

OPENAI_API_KEY sk-proj-pw


# Prompt Template

In [2]:
from langchain_core.prompts import PromptTemplate

In [5]:
template ="What is a good name for a company that makes {product}?"
prompt = PromptTemplate.from_template(template)

In [6]:
formatted_prompt = prompt.format(product="colorful socks")
print(formatted_prompt)

What is a good name for a company that makes colorful socks?


# ChatOpenAI

In [7]:
from langchain_openai import ChatOpenAI

In [8]:
llm = ChatOpenAI(model="gpt-4.1-nano", max_tokens=50, temperature=0.2)

In [9]:
response = llm.invoke([("human", "Hello, who are you?")])
print(response)
print(response.content)

content="Hello! I'm an AI language model here to assist you. How can I help today?" additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 13, 'total_tokens': 31, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_38343a2f8f', 'id': 'chatcmpl-Bk2PQwSZEiNNfSuyYbpjhp6J0SjcW', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--a018710a-80ed-4d2d-8084-654df1cb30f8-0' usage_metadata={'input_tokens': 13, 'output_tokens': 18, 'total_tokens': 31, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
Hello! I'm an AI language model here to assist you. How can I help today?


# Runnabel
Runnabel 인터페이스 : 실행가능 구성 요소. 서로 다른 컴포넌트들이 호환되는 형태로 결합되게

# StrOutputParser

In [10]:
from langchain_core.output_parsers import StrOutputParser

In [11]:
parser = StrOutputParser()
parsed_response = parser.invoke(response)
print(parsed_response)

Hello! I'm an AI language model here to assist you. How can I help today?


# Pipeline, LCEL (Langchain Expression Language)

In [12]:
chain = prompt | llm | parser 

In [13]:
result = chain.invoke({"product" : "coffee"})
print(result)

Sure! Here are some creative and appealing name ideas for a coffee company:

1. Brew Haven  
2. Java Junction  
3. Morning Mug  
4. Bean & Bliss  
5. Rise & Roast  
6. Caffeine Craft  
7.


# Langchain으로 토큰 사용량 추적

In [14]:
print("토큰 사용 정보 : ", response.usage_metadata)

토큰 사용 정보 :  {'input_tokens': 13, 'output_tokens': 18, 'total_tokens': 31, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [15]:
!uv add langchain_community

Resolved 58 packages in 557ms
 Downloaded langchain-community
Prepared 14 packages in 5.15s
Installed 15 packages in 705ms
 + aiohappyeyeballs==2.6.1
 + aiohttp==3.12.13
 + aiosignal==1.3.2
 + dataclasses-json==0.6.7
 + frozenlist==1.7.0
 + httpx-sse==0.4.0
 + langchain-community==0.3.25
 + marshmallow==3.26.1
 + multidict==6.5.0
 + mypy-extensions==1.1.0
 + numpy==2.3.0
 + propcache==0.3.2
 + pydantic-settings==2.9.1
 + typing-inspect==0.9.0
 + yarl==1.20.1


In [16]:
# 콜백을 통한 누적 토큰 추적
from langchain_community.callbacks import get_openai_callback

In [19]:
with get_openai_callback() as cb:
    res1 = llm.invoke("조치원의 오늘 날씨는?")
    print(f"응답: {res1.content[:30]}...")

    res2 = llm.invoke("파이썬으로 리스트 만드는 법?")
    print(f"응답: {res2.content[:30]}...")


응답: 죄송합니다만, 현재 실시간 날씨 정보를 제공할 수 없습...
응답: 파이썬에서 리스트를 만드는 방법은 여러 가지가 있습니다...


In [20]:
    # 토큰 및 비용 출력
    print("\n--- 사용량 요약 ---")
    print(f"총 토큰 수: {cb.total_tokens}")
    print(f"프롬프트 토큰 수: {cb.prompt_tokens}")
    print(f"응답 토큰 수: {cb.completion_tokens}")
    print(f"총 비용 (USD): ${cb.total_cost:.6f}")


--- 사용량 요약 ---
총 토큰 수: 123
프롬프트 토큰 수: 31
응답 토큰 수: 92
총 비용 (USD): $0.000040


# LangChain LLM 응답 캐싱

In [21]:
from langchain_core.caches import InMemoryCache
from langchain_core.globals import set_llm_cache
# InMemoryCache 설정
set_llm_cache(InMemoryCache())

In [ ]:
query = "재밌는 유머 추천"
result1 = llm.invoke(query)
print("응답1: ", result1.content)

응답2:  물론이죠! 여기 재미있는 유머 하나 소개할게요:

왜 컴퓨터는 피곤하지 않을까?  
왜냐하면 항상 휴식을 취하는 ‘윈도우’가 있기 때문이야!  

더 웃긴


In [ ]:
result2 = llm.invoke(query)
print("응답2: ", result2.content)

응답2:  물론이죠! 여기 재미있는 유머 하나 소개할게요:

왜 컴퓨터는 피곤하지 않을까?  
왜냐하면 항상 휴식을 취하는 ‘윈도우’가 있기 때문이야!  

더 웃긴


In [26]:
# SQLite 를 이용 디스크 기반 캐시 설정
import os
from langchain_community.cache import SQLiteCache

In [28]:
# 기존 캐시 DB 삭제 (실습용)
if os.path.exists(".langchain_cache.db"):
    os.remove(".langchain_cache.db")